<a href="https://colab.research.google.com/github/young-hyun-park/capston_design/blob/main/resnet151_multilabelclassfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import torch
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image
import os
import re
import nibabel as nib

## Data Load

In [ ]:
zip_path = sorted(glob('/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/*'))

In [ ]:
import zipfile
for path in zip_path:
  zip_file = zipfile.ZipFile(path)
  zip_file.extractall('/content/datasets/'+path.split('/')[-1].split('.')[0])
  zip_file.close()

In [ ]:
PATH = '/content/datasets/'
from glob import glob
file_data = glob(PATH+'*')

In [ ]:
my_glob = glob('/content/datasets/images*/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  76666


## Data Load

In [ ]:
import pandas as pd
df= pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/processed_data_entry.csv')

In [ ]:
df.drop(['Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]'],axis = 1,inplace = True)

In [ ]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

In [ ]:
classes = ['Atelectasis',
                'Consolidation',
                'Infiltration', 
                'Pneumothorax', 
                'Edema', 
                'Emphysema',
                'Fibrosis', 
                'Effusion',
                'Pneumonia',
                'Pleural_Thickening',
                'Cardiomegaly',
                'Nodule', 
                'Mass', 
                'Hernia']
for label in classes:
  df[label] = df['Finding Labels'].map(lambda result: 1.0 if label in result else 0)

In [ ]:
data = df[classes]
df['labels'] = data.apply(lambda row: np.argmax(row) if np.sum(row)>0 else -1, axis = 1)

In [ ]:
df = df[df['Finding Labels']!= 'No Finding']
df.reset_index(drop = True, inplace = True)

In [ ]:
df.columns

Index(['Image Index', 'Finding Labels', 'full_path', 'Atelectasis',
       'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema',
       'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
       'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'labels'],
      dtype='object')

In [ ]:
target_cols = df.drop(['Image Index','Finding Labels', 'full_path','labels'], axis=1).columns.to_list()

In [ ]:
target_cols

['Atelectasis',
 'Consolidation',
 'Infiltration',
 'Pneumothorax',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Effusion',
 'Pneumonia',
 'Pleural_Thickening',
 'Cardiomegaly',
 'Nodule',
 'Mass',
 'Hernia']

In [ ]:
from sklearn.model_selection import train_test_split
paths = df['full_path'].values
columns = df.columns[3:-1]
labels = df[columns].values
(train_path, val_path ,train_labels , val_labels) = train_test_split(paths,labels,test_size = 0.2,random_state = 42)

In [ ]:
val_labels[1]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0.])

In [ ]:
class Train_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img = img[:,:,np.newaxis] 
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
class val_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img = img[:,:,np.newaxis]    
        label = self.labels [idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
#!pip install --upgrade --force-reinstall --no-deps albumentations
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 5.1 MB/s 
     |████████████████████████████████| 948 kB 58.7 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=08cf2695da239ab88ae4fedea2c2b80eeed7a6381cb44b133785cb0fed20fb82
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import albumentations as A                                                                           
from albumentations.pytorch import ToTensorV2

In [ ]:
data_transforms = {
    'train': A.Compose(
    [
     ToTensorV2()
     ]
    ),
    'val': A.Compose(
        [
     ToTensorV2()
                            ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]

# Number of classes in the dataset
num_classes = 14

# Batch size for training (change depending on how much memory you have)
batch_size = 2

# Number of epochs to train for
num_epochs = 5

In [ ]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [ ]:
image_datasets = {'train' : train_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataloaders_dict['train']

## Model

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
pip install timm

     |████████████████████████████████| 431 kB 5.1 MB/s 


In [ ]:
import timm

In [ ]:
model = torch.load('/content/drive/Shareddrives/캡스톤 디자인1/codes/resnet151_multilabelclassfication31_35epoch.pt')

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , roc_curve , accuracy_score,classification_report,multilabel_confusion_matrix
import tqdm

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model.to(device)

criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer_ft = optim.Adam(model.parameters(), lr=0.0001)
sgdr_partial = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=5, eta_min=0.005 )

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('|{}Epoch {}phase multilabel_confusion_matrix report|'.format(epoch+1,phase))
            print('--------------------------------')
            print('\n{}'.format(epoch_multilabel_confusion_matrix))
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_hist.append(epoch_loss)
                epoch_val_label.append(full_label)
                epoch_val_output.append(full_prediction)
                val_classification_report.append(epoch_classification_report)
                val_confusion_matrix.append(multilabel_confusion_matrix)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report,'val_label' : epoch_val_label , 'Confusion_matrix' :val_confusion_matrix,'val_output' : epoch_val_output}
    #, , 
    return model, train_dict ,val_dict

In [ ]:
#summary(model.to('cuda'),(3,512,512))

In [ ]:
criterion = nn.BCEWithLogitsLoss()
import pickle
# Train and evaluate
for i in range(3):
  print('\n--------------------------------')
  print('|        Epoch {} ~ {}      |'.format(i*5+36, (i+1)*5+35))
  print('\n--------------------------------')
  model_ft, train_dict, val_dict  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=num_epochs)
  torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/resnet151_multilabelclassfication{}_{}epoch.pt'.format(i*5+31, (i+1)*5+30))
  with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/resnet151_multilabelclassfication{}_{}epoch_train_dict.pkl'.format(i*5+31, (i+1)*5+30),'wb') as fw:
    pickle.dump(train_dict, fw)
  with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/resnet151_multilabelclassfication{}_{}epoch_val_dict.pkl'.format(i*5+31, (i+1)*5+30),'wb') as ff:
    pickle.dump(val_dict, ff)
'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''


--------------------------------
|        Epoch 36 ~ 40      |

--------------------------------
Epoch 1/5
----------
phase : 'train'


  0%|          | 0/20704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train Loss: 0.2248 Acc: 0.2301, Time : 6460.9629

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[30574  1603]
  [ 5736  3494]]

 [[37631    24]
  [ 3729    23]]

 [[21489  4043]
  [ 8124  7751]]

 [[36490   633]
  [ 2681  1603]]

 [[39464   121]
  [ 1635   187]]

 [[39260   175]
  [ 1607   365]]

 [[40049     4]
  [ 1352     2]]

 [[28812  1951]
  [ 4529  6115]]

 [[40268     0]
  [ 1139     0]]

 [[38628    56]
  [ 2654    69]]

 [[38905   284]
  [  988  1230]]

 [[36175   165]
  [ 4852   215]]

 [[36159   609]
  [ 3034  1605]]

 [[41224     1]
  [  160    22]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

       Atelectasis       0.69      0.38      0.49      9230
     Consolidation       0.49      0.01      0.01      3752
      Infiltration       0.66      0.49      0.56     15875
  

  0%|          | 0/5176 [00:00<?, ?it/s]

...


train Loss: 0.2154 Acc: 0.2544, Time : 6493.4217

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[30561  1616]
  [ 5334  3896]]

 [[37614    41]
  [ 3715    37]]

 [[21514  4018]
  [ 7646  8229]]

 [[36448   675]
  [ 2421  1863]]

 [[39426   159]
  [ 1574   248]]

 [[39211   224]
  [ 1521   451]]

 [[40043    10]
  [ 1346     8]]

 [[28902  1861]
  [ 4154  6490]]

 [[40268     0]
  [ 1139     0]]

 [[38583   101]
  [ 2594   129]]

 [[38919   270]
  [  884  1334]]

 [[36079   261]
  [ 4758   309]]

 [[36147   621]
  [ 2734  1905]]

 [[41219     6]
  [  155    27]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

       Atelectasis       0.71      0.42      0.53      9230
     Consolidation       0.47      0.01      0.02      3752
      Infiltration       0.67      0.52      0.59     15875
  

  0%|          | 0/5176 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



val Loss: 0.3030 Acc: 0.1547, Time : 444.4502

--------------------------------
|1Epoch valphase multilabel_confusion_matrix report|
--------------------------------

[[[ 7404   619]
  [ 1653   676]]

 [[ 9415    22]
  [  910     5]]

 [[ 4936  1397]
  [ 2410  1609]]

 [[ 9133   201]
  [  855   163]]

 [[ 9769   102]
  [  429    52]]

 [[ 9742    66]
  [  497    47]]

 [[10017     3]
  [  330     2]]

 [[ 6988   691]
  [ 1532  1141]]

 [[10050    10]
  [  292     0]]

 [[ 9665    25]
  [  652    10]]

 [[ 9559   235]
  [  380   178]]

 [[ 9074    14]
  [ 1257     7]]

 [[ 8987   222]
  [  966   177]]

 [[10301     6]
  [   45     0]]]

--------------------------------

1Epoch valphase classification report
--------------------------------
                    precision    recall  f1-score   support

       Atelectasis       0.52      0.29      0.37      2329
     Consolidation       0.19      0.01      0.01       915
      Infiltration       0.54      0.40      0.46      4019
      Pne

  0%|          | 0/20704 [00:00<?, ?it/s]

In [ ]:
train_loss = []
val_loss = []
for i in range(len(train_dict['Loss'])):
  train_loss.append(train_dict['Loss'][i])
for i in range(len(val_dict['Loss'])):
  val_loss.append(val_dict['Loss'][i])

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('Resnet152 Loss', fontsize = 15)
plt.xlabel('Epoch', fontsize = 15)
plt.ylabel('Loss', fontsize = 15)
plt.ylim(0.2,0.3)
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
train_accuracy = []
val_accuracy = []
for i in range(len(train_dict['Accuarcy'])):
  train_accuracy.append(train_dict['Accuarcy'][i].to('cpu').numpy())
for i in range(len(val_dict['Accuarcy'])):
  val_accuracy.append(val_dict['Accuarcy'][i].to('cpu').numpy())

In [ ]:
plt.plot(train_accuracy)
plt.plot(val_accuracy)
plt.title('Resnet152 Accuracy', fontsize = 15)
plt.xlabel('Epoch', fontsize = 15)
plt.ylabel('Accuracy', fontsize = 15)
plt.ylim(0,1)
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
model_ft = torch.load('/content/drive/Shareddrives/캡스톤 디자인1/codes/resnet151_multiclassclassfication_nofinding_제외_timm.pt')

In [ ]:
model_ft = model_ft.to(device)


# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(),lr=0.001)# 업데이트 할 파라미터만 넣어준다.
class test_Dataset(Dataset):
    def __init__(self, data_path,transform = None):
        self.data_path = data_path
        self.transform = transform
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img[:,:,np.newaxis]
        img = img/255
        label =  class2idx[df[df['Image Index']== path.split('/')[-1]]['Finding Labels'].values[0]]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image.float(), label

In [ ]:
data_transforms_test = A.Compose(
        [
     ToTensorV2()
     ]
     )

In [ ]:
test_data = test_Dataset(val_path,transform = data_transforms_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [ ]:
summary(model_ft.to('cuda'),(1,512,512))

In [ ]:
running_loss = 0.0
running_corrects = 0
start = time.time()
output_list = list()
criterion = nn.CrossEntropyLoss()

for inputs, labels in test_loader:
    inputs = inputs.to(device = device, dtype = torch.float32)
    labels = labels.to(device = device)
    # zero the parameter gradients
    optimizer_ft.zero_grad()
    outputs = model_ft(inputs)
    output_list+=list((torch.argmax(outputs,axis = 1).cpu().numpy()))
    loss = criterion(outputs, labels)
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_acc = running_corrects.double() / len(test_loader.dataset)
print('Test Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(epoch_loss, epoch_acc, time.time()-start))

In [ ]:
len(test_loader)*2

In [ ]:
labels

In [ ]:
torch.argmax(outputs,axis = 1).cpu().numpy()

In [ ]:
outputs.shape

In [ ]:
dummy_df = pd.DataFrame(output_list)

In [ ]:
dummy_df.value_counts()

In [ ]:
val_df['Finding Labels'].value_counts()

In [ ]:
class2idx

In [ ]:
#레이어 시각화

for w in model_ft.parameters():
    w = w.data.cpu()
    print(w.shape)
    break

# normalize weights
min_w = torch.min(w)
w1 = (-1/(2 * min_w)) * w + 0.5

# make grid to display it
grid_size = len(w1)
x_grid = [w1[i] for i in range(grid_size)]
x_grid = torchvision.utils.make_grid(x_grid, nrow=8, padding=1)

plt.figure(figsize=(10, 10))
plt.imshow(x_grid.permute(2,1,0))

In [ ]:
sample_image = np.array(Image.open(train_path[0]))
sample_image = sample_image[:,:,np.newaxis]/255
transform = transforms.Compose([
    transforms.ToTensor()
])
sample_image = transform(sample_image)
sample_image=sample_image.unsqueeze(0)

In [ ]:
sample_image

In [ ]:
no_of_layers=0
conv_layers=[]

model_children=list(model_ft.children())

for child in model_children:
  if type(child)==nn.Conv2d:
    no_of_layers+=1
    conv_layers.append(child)
  elif type(child)==nn.Sequential:
    for layer in child.children():
      for bottleneck in layer.children():
        if type(bottleneck)==nn.Conv2d:
          no_of_layers+=1
          conv_layers.append(bottleneck)
print(no_of_layers)

In [ ]:
results = [conv_layers[0](sample_image.to(device = device, dtype = torch.float32))]
for i in range(1, len(conv_layers)):
    results.append(conv_layers[i](results[-1]))
outputs = results

In [ ]:
# visualize 8 features map from each layer 
for num_layer in range(len(results)):
    plt.figure(figsize=(50, 10))
    layer_viz = results[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print("Layer ",num_layer+1)
    for i, filter in enumerate(layer_viz):
        if i == 16: 
            break
        plt.subplot(2, 8, i + 1)
        plt.imshow(filter.cpu().numpy(), cmap='gray')
        plt.axis("off")
    plt.show()
    plt.close()